In [377]:
import numpy as np
import copy

class Electre:
  def __init__(self, bobot , alternatif , alternatif_idx):
    self.alternatif_idx = alternatif_idx
    self.bobot = bobot
    self.alternatif = alternatif
    self.norm_alternatif = copy.copy(self.alternatif)
    self.norm_alternatif[:] = 0
    self.val_mat_v = copy.copy(self.alternatif)
    self.val_mat_v[:] = 0
    self.new_idx_val = self.alternatif.shape[0] ** 2 - self.alternatif.shape[0]
    self.search_idx = np.ones((self.new_idx_val,self.alternatif.shape[1]), dtype=bool)
    self.matrix_concordence = np.zeros((self.alternatif.shape[0],self.alternatif.shape[0]), dtype=float)
    self.matrix_discordence = np.zeros((self.alternatif.shape[0],self.alternatif.shape[0]), dtype=float)
    self.matrix_dom_con = np.zeros((self.alternatif.shape[0],self.alternatif.shape[0]), dtype=float)
    self.matrix_dom_dis = np.zeros((self.alternatif.shape[0],self.alternatif.shape[0]), dtype=float)
    self.result_agregrasi = np.zeros((self.alternatif.shape[0],self.alternatif.shape[0]), dtype=float)
    self.treshold_concordence = np.array([0],dtype=float)
    self.treshold_discordinance = np.array([0],dtype=float)
    self.new_inc = 0
    self.inc_val = 0
    self.pos_idx = []
    self.idx_val = []
    self.temp_dic = []
    self.him_idx_con = []
    self.him_idx_dis = []
    self.normalisasi()

  def normalisasi(self):
    trans_alternatif = self.alternatif.T
    self.norm_alternatif = self.norm_alternatif.T
    for i in range(0,trans_alternatif.shape[0]):
      for j in range(0,trans_alternatif.shape[1]):
        val = np.sum(trans_alternatif[i] ** 2)
        self.norm_alternatif[i,j] = trans_alternatif[i,j] / np.sqrt(val)

    self.norm_alternatif = self.norm_alternatif.T
    self.matriks_v()

  def matriks_v(self):
    for i in range(self.norm_alternatif.shape[0]):
      self.val_mat_v[i] = self.norm_alternatif[i] * self.bobot
  
    self.idx_get_concordance()

  def idx_get_concordance(self):
    for i in range(0,self.alternatif.shape[0]):
      self.compute_search_idx(i)

  def compute_search_idx(self,pos):
    for i in range(0,self.alternatif.shape[0]):
      if(pos != i):
        self.search_idx[i] = self.val_mat_v[pos] >= self.val_mat_v[i]
        self.get_idx(self.search_idx[i], pos , i)
  def get_idx(self,val, pos , i):
    # print(pos,i)
    self.new_inc += 1
    self.pos_idx.append(pos)
    self.idx_val.append(i)
    idx_concordence = np.where(val == True)
    idx_disconcordence = np.where(val == False)
    self.him_idx_con.append(idx_concordence)
    self.him_idx_dis.append(idx_disconcordence)
    self.compute_matrix_discordence(idx_disconcordence)

    # print(idx_concordence,idx_disconcordence)
    for j in range(0,self.alternatif.shape[1]):
      # self.matrix_concordence[pos,i] = sum(idx_concordence[j])
      new_sum = 0
      for k in range(0,len(idx_concordence[j])):
        new_sum += self.bobot[idx_concordence[j][k]]
      self.matrix_concordence[pos,i] = new_sum
      #print(idx_concordence[j])
      break
    # print(self.matrix_concordence)
    self.treshold_concordence[0] = np.sum(self.matrix_concordence / (self.alternatif.shape[0] * (self.alternatif.shape[0] - 1)))
    if(self.new_inc == len(self.search_idx)):
      self.get_dominan_matrix_concordance(self.pos_idx,self.idx_val)
      self.new_inc = 0
    return self.treshold_concordence
  
  def get_dominan_matrix_concordance(self,pos,i):
    # print(self.norm_alternatif)
    # print(self.val_mat_v)
    # print(self.matrix_concordence)
    # print(self.treshold_concordence)
    for k in range(0,self.new_idx_val):
      if(self.matrix_concordence[pos[k],i[k]] >= self.treshold_concordence[0]):
        self.matrix_dom_con[pos[k],i[k]] = 1
      # pass
    # print(self.matrix_dom_con)

    self.compute_matrix_agregrasi()

  def compute_matrix_discordence(self,val):
    self.temp_dic.append(val)
    if(self.new_inc == len(self.search_idx)):
      # for i in range(0,len(self.idx_val)):
      idx_new = np.asarray((self.temp_dic))
      for i in range(idx_new.shape[0]):
        for j in range(idx_new.shape[1]):
            # print(idx_new[i][j] , self.pos_idx,self.idx_val)
            self.get_matrix_discordence(idx_new[i][j] , self.pos_idx[i],self.idx_val[i], i)
      # pass 
    return self.new_inc

  def get_matrix_discordence(self, idx , al1 , al2, inc):
    self.inc_val = inc
    if(len(idx) != 0):
      pem_idx = np.arange(0,self.val_mat_v.shape[1])
      pem_rs = pem_idx.reshape(len(pem_idx),1)
      pembagi = abs(self.val_mat_v[al1,pem_rs] - self.val_mat_v[al2,pem_rs])
      pembagi_max = np.max(pembagi)
      
      new_shp = idx.reshape(len(idx),1)
      new_val = abs(self.val_mat_v[al1,new_shp] - self.val_mat_v[al2,new_shp])
      max_val = np.max(new_val)

      res = max_val / pembagi_max
      self.matrix_discordence[al1,al2] = res
      self.treshold_discordinance[0] = np.sum(self.matrix_discordence / (self.alternatif.shape[0] * (self.alternatif.shape[0] - 1)))
    
    if(self.inc_val + 1 == self.search_idx.shape[0]):
      self.calculate_matrix_dominan_discordenca()
      # pass
    
  def calculate_matrix_dominan_discordenca(self):
    for k in range(0,self.new_idx_val):
      if(self.matrix_discordence[self.pos_idx[k],self.idx_val[k]] >= self.treshold_discordinance[0]):
        self.matrix_dom_dis[self.pos_idx[k],self.idx_val[k]] = 1
    
    return self.matrix_dom_dis

  def compute_matrix_agregrasi(self):
    self.rank = np.zeros((self.matrix_dom_dis.shape[0], 1))
    self.result_agregrasi = self.matrix_dom_dis * self.matrix_dom_con
    for i in range(self.result_agregrasi.shape[0]):
      for j in range(self.result_agregrasi.shape[1]):
        if(self.result_agregrasi[i][j] == 1):
          self.rank[i] += 1

    new_rank = np.array([[0,""]]*self.rank.shape[0])
    for i in range (0,new_rank.shape[0]):
        new_rank[i][0] = self.rank[i][0]
        new_rank[i][1] = self.alternatif_idx[i][0]

    print("Bobot : ")
    print(self.bobot)
    print("\n---------------------------------------------------\n")
    
    print("Kriteria : ")
    print(self.alternatif)
    print("\n---------------------------------------------------\n")

    print("Matriks Normalisasi : ")
    print(self.norm_alternatif)
    print("\n---------------------------------------------------\n")

    print("Matriks V : ")
    print(self.val_mat_v)
    print("\n---------------------------------------------------\n")

    print("Himpunan Concordance : ")
    print(self.him_idx_con)
    print("\n---------------------------------------------------\n")

    print("Himpunan Disconcordance : ")
    print(self.him_idx_dis)
    print("\n---------------------------------------------------\n")


    print("Matriks Concordance : ")
    print(self.matrix_concordence)
    print("\n---------------------------------------------------\n")

    print("Matriks Disconcordance : ")
    print(self.matrix_discordence)
    print("\n---------------------------------------------------\n")

    print("Treshold Concordance : ")
    print(self.treshold_concordence)
    print("\n---------------------------------------------------\n")

    print("Treshold Disconcordance : ")
    print(self.treshold_discordinance)
    print("\n---------------------------------------------------\n")

    print("Matriks Dominan Concordance : ")
    print(self.matrix_dom_con)
    print("\n---------------------------------------------------\n")

    print("Matriks Dominan Disconcordance : ")
    print(self.matrix_dom_dis)
    print("\n---------------------------------------------------\n")

    print("Matriks Agregrasi : ")
    print(self.result_agregrasi)
    print("\n---------------------------------------------------\n")

    print("Ranking Before Sort: ")
    print(new_rank)
    print("\n---------------------------------------------------\n")

    print("Ranking After Sort: ")
    sort_rank = new_rank[new_rank[:,0].argsort()[::-1]]
    print(sort_rank)
    print("\n---------------------------------------------------\n")
    
if __name__ == '__main__':
  try:
    # bobot = np.array([3,2,2,2,1])
    # alternatif = np.array([[2,4,2,3,3],[4,1,5,5,3],[3,2,1,4,4]], dtype=float)
    # alternatif_idx = np.array([["Apartemen 1"],["Apartemen 2"],["Apartemen 3"]])
    # bobot = np.array([5 ,2 ,4 ,2])
    # alternatif = np.array([[5 ,3 ,3 ,3],[4 ,4 ,3 ,4],[4 ,4 ,3 ,3],[2 ,4 ,3 ,3]], dtype=float)
    # alternatif_idx = np.array([["Apartemen 1"],["Apartemen 2"],["Apartemen 3"],["Apartemen 4"]])

    bobot = np.array([1 ,3 ,5 ,1])
    alternatif = np.array([[1 ,3 ,3 ,2],[2 ,2 ,3 ,2],[2 ,3 ,3 ,2],[1 ,3 ,3 ,2]], dtype=float)
    alternatif_idx = np.array([["A1"],["A2"],["A3"],["A4"]])

    if(bobot.shape[0] == alternatif.shape[1] and alternatif.shape[0] == alternatif_idx.shape[0]):
      el = Electre(bobot,alternatif,alternatif_idx)
    else:
      print ("Shape Alternatif and Bobot not same")
  except Exception as e:
    print(e)

Bobot : 
[1 3 5 1]

---------------------------------------------------

Kriteria : 
[[1. 3. 3. 2.]
 [2. 2. 3. 2.]
 [2. 3. 3. 2.]
 [1. 3. 3. 2.]]

---------------------------------------------------

Matriks Normalisasi : 
[[0.31622777 0.53881591 0.5        0.5       ]
 [0.63245553 0.3592106  0.5        0.5       ]
 [0.63245553 0.53881591 0.5        0.5       ]
 [0.31622777 0.53881591 0.5        0.5       ]]

---------------------------------------------------

Matriks V : 
[[0.31622777 1.61644772 2.5        0.5       ]
 [0.63245553 1.07763181 2.5        0.5       ]
 [0.63245553 1.61644772 2.5        0.5       ]
 [0.31622777 1.61644772 2.5        0.5       ]]

---------------------------------------------------

Himpunan Concordance : 
[(array([1, 2, 3]),), (array([1, 2, 3]),), (array([0, 1, 2, 3]),), (array([0, 2, 3]),), (array([0, 2, 3]),), (array([0, 2, 3]),), (array([0, 1, 2, 3]),), (array([0, 1, 2, 3]),), (array([0, 1, 2, 3]),), (array([0, 1, 2, 3]),), (array([1, 2, 3]),), (array(